In [1]:
# Load Biospytial modules and etc.
%matplotlib inline
import sys
sys.path.append('/apps')
import django
django.setup()
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
## Use the ggplot style
plt.style.use('ggplot')

In [ ]:
## Sectionate by coordinates
##minx = -85
##maxx = -80
##miny = 30
##maxy = 35


### PATH information for running in the Biospytial container
#plotdata_path = "/RawDataCSV/idiv_share/plotsClimateData_11092017.csv"
#empirical_data_path = "/apps/external_plugins/spystats/HEC_runs/results/logbiomas_logsppn_res.csv"


In [2]:
run ../HEC_runs/fit_fia_logbiomass_logspp_GLS.py /RawDataCSV/idiv_share/plotsClimateData_11092017.csv /apps/external_plugins/spystats/HEC_runs/results/logbiomas_logsppn_res.csv -85 -80 30 35

/opt/conda/envs/biospytial/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2481: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  self.compile if kw['shell_futures'] else None)
INFO:__main__:Reprojecting to Alberts equal area
INFO:__main__:Fitting OLS linear model: logBiomass ~ logSppN 
INFO:__main__:Removing possible duplicates. 
 This avoids problems of Non Positive semidefinite
INFO:__main__:Reading the empirical Variogram file
INFO:__main__:Instantiating a Variogram object with the values calculated before
INFO:__main__:Dropping possible Nans
INFO:__main__:Instantiating Matern Model...
INFO:__main__:fitting Matern Model with the empirical variogram
../tools.py:510: RuntimeWarning: divide by zero encountered in power
  rho_h = a * np.power(b,kappa) * K_v_b
/opt/conda/envs/biospytial/lib/python2.7/site-packages/scipy/optimize/minpack.py:690: OptimizeWarning: Covariance of the parameters could not be estimated
  catego

In [4]:
data = results['dataframe']

In [5]:
data.shape

(36844, 47)

In [6]:
data.columns

Index([    u'Unnamed: 0',        u'STATECD',        u'STATENM',
              u'STATEAB',       u'COUNTYCD',           u'PLOT',
              u'PLOT_ID',        u'PlotIDn',            u'LAT',
                  u'LON',          u'INVYR',      u'N_subplot',
             u'StandAge',    u'treeDensity',         u'plotBA',
          u'plotBiomass',           u'SppN',       u'ECO_ID_U',
             u'ECO_CODE',       u'ECO_NAME',        u'ECO_NUM',
           u'ECODE_NAME',       u'CLS_CODE',      u'ECO_NOTES',
            u'WWF_REALM',     u'WWF_REALM2',     u'WWF_MHTNUM',
           u'WWF_MHTNAM',       u'RealmMHT',      u'ER_UPDATE',
            u'ER_DATE_U',      u'ER_RATION',     u'SOURCEDATA',
                 u'ECOR', u'AnnualMeanPrec',     u'TempSeason',
           u'AnnualPrec',     u'PrecSeason',            u'AET',
                  u'lon',            u'lat',       u'geometry',
           u'logBiomass',         u'newLon',         u'newLat',
              u'logSppN',      u'residua

In [8]:
cd /outputs/


/outputs


In [9]:
ls

My_first_GaussianSim.png      covariance_mat_gp1.png  idiv_scenarios/
bursera_sampling_100_env.csv  gt_tree.png


In [11]:
data.to_csv("reproj_plotsClimateData_11092017.csv")